Paso 0 — Reset seguro de la colección (opcional)

In [1]:
# --- Paso 0: Reset opcional de la colección ---

import os, chromadb
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

VDB_PATH = "./vectordb"
COLLECTION_NAME = "autoselx_docs"

def reset_collection(full_reset=False, collection_name=COLLECTION_NAME):
    import chromadb
    if full_reset:
        if os.path.exists(VDB_PATH):
            shutil.rmtree(VDB_PATH)
            print(f"🧹 Directorio {VDB_PATH} borrado por completo.")
        os.makedirs(VDB_PATH, exist_ok=True)
    else:
        client = chromadb.PersistentClient(path=VDB_PATH)
        try:
            client.delete_collection(name=collection_name)
            print(f"🧹 Colección '{collection_name}' eliminada.")
        except Exception as e:
            print(f"(info) No se eliminó colección: {e}")


In [ ]:
# --- Paso 1: imports y utilidades ---
import os, re, pdfplumber
from typing import List, Tuple, Dict
from collections import Counter

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document   # ✅ Import correcto

from dotenv import load_dotenv
import os

# Cargar variables del archivo .env
load_dotenv()

# Leer la API key desde el entorno
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


# --- Patrones de limpieza (encabezados/pies repetidos) ---
CLEAN_PATTERNS = [
    r"FORMATO REQUISICIÓN.*",
    r"ECOPETROL DESARROLLO DE PROYECTO.*",
    r"Todos los derechos reservados.*",
    r"EDP-F-046.*Versión.*",
    r"MR.*#:.*CAS.*",
    r"Página\s+\d+\s+de\s+\d+",
    r"_{6,}",  # líneas largas
]

def clean_text(text: str) -> str:
    for p in CLEAN_PATTERNS:
        text = re.sub(p, "", text, flags=re.IGNORECASE)
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()

# --- Palabras clave para asignar prioridad ---
HD_HINTS = [
    r"\bAPI\s*675\b", r"HOJA\s*DE\s*DATOS", r"\bDATA\s*SHEET\b", r"\bDATASHEET\b",
    r"BOMB(A|S)?\s+(DOSIFICADORA|DOSING)"
]

ALCANCE_HINTS = [
    r"\bALCANCE\b", r"ALCANCE\s+DEL\s+SUMINISTRO", r"\bSCOPE\b", r"SCOPE\s+OF\s+SUPPLY"
]

REQ_HINTS = [
    r"\bREQUISITO(S)?\b", r"\bREQUERIMIENTO(S)?\b",
    r"REQUISITO(S)?\s+T(É|E)CNICO(S)?", r"CONDICIONES?\s+T(É|E)CNICAS?",
    r"CRITERIOS?\s+DE\s+DISE(Ñ|N)O", r"DATOS?\s+DE\s+DISE(Ñ|N)O",
    r"\bREQUIREMENT(S)?\b", r"DESIGN\s+REQUIREMENT(S)?\b",
    r"TECHNICAL\s+REQUIREMENT(S)?\b", r"\bSPECIFICATION(S)?\b",
    r"MINIMUM\s+REQUIREMENT(S)?\b",
    r"\bEL\s+PROVEEDOR\s+DEBE\b", r"\bEL\s+VENDEDOR\s+DEBE\b",
    r"\bDEBERÁ\b", r"\bSE\s+REQUIERE(N)?\b",
    r"\bSHALL\b", r"\bMUST\b", r"\bIS\s+REQUIRED\s+TO\b"
]

NORM_HINTS = [
    r"\bNORMAS?\b", r"\bEST[ÁA]NDA(RE|R)E?S?\b",
    r"\bC(Ó|O)DIGO(S)?\b", r"\bCODE(S)?\b", r"\bSTANDARD(S)?\b",
    r"\bSPEC(S|IFICATIONS)?\b", r"\bREGULATION(S)?\b",
    r"\bRETIE\b", r"\bAPI\b", r"\bASTM\b", r"\bASME\b", r"\bIEC\b", r"\bIEEE\b",
    r"APPLICABLE\s+(CODES|STANDARDS|SPECIFICATIONS)"
]

def matches_any(text: str, patterns: List[str]) -> bool:
    return any(re.search(p, text, flags=re.IGNORECASE) for p in patterns)

def detect_priority(text: str) -> str:
    """Clasifica el chunk en una categoría fija de prioridad"""
    if matches_any(text, HD_HINTS):      return "hoja_datos"
    if matches_any(text, ALCANCE_HINTS): return "alcance"
    if matches_any(text, REQ_HINTS):     return "requisito"
    if matches_any(text, NORM_HINTS):    return "norma"
    return "general"

# --- Conversión de tablas ---
def table_to_text_and_kv(rows: List[List[str]]) -> Tuple[str, Dict[str, str] | None]:
    clean_rows = []
    for r in rows:
        if not r:
            continue
        rr = [(c or "").strip() for c in r]
        if any(rr):
            clean_rows.append(rr)

    kv = None
    if clean_rows and max(len(r) for r in clean_rows) <= 2 and len(clean_rows) >= 2:
        kv = {}
        for r in clean_rows:
            if len(r) >= 2 and r[0] and r[1]:
                kv[r[0]] = r[1]

    text = "\n".join(" | ".join(r) for r in clean_rows)
    return text, kv if kv else None

# --- KV extractor unificado (texto + tablas) ---
def kv_from_text_page(text: str) -> Dict[str, str]:
    """
    Extrae pares clave-valor típicos de Hoja de Datos (texto o tabla).
    Soporta caudal, presión, potencia, viscosidad, redundancia, materiales, API 675.
    """
    t = " ".join(text.split()).upper()
    kv = {}

    # Caudal (GPH o LPH, con rangos)
    m = re.search(r"(\d+(?:[.,]\d+)?)\s*(?:-|–|A|TO)?\s*(\d+(?:[.,]\d+)?)?\s*GPH", t)
    if m: kv["caudal_gph"] = f"{m.group(1)}-{m.group(2)}" if m.group(2) else m.group(1)

    m = re.search(r"(\d+(?:[.,]\d+)?)\s*(?:-|–|A|TO)?\s*(\d+(?:[.,]\d+)?)?\s*LPH", t)
    if m: kv["caudal_lph"] = f"{m.group(1)}-{m.group(2)}" if m.group(2) else m.group(1)

    # Presión (PSI o BAR, con rangos)
    m = re.search(r"(\d+(?:[.,]\d+)?)\s*(?:-|–|A|TO)?\s*(\d+(?:[.,]\d+)?)?\s*PSI", t)
    if m: kv["presion_psig"] = f"{m.group(1)}-{m.group(2)}" if m.group(2) else m.group(1)

    m = re.search(r"(\d+(?:[.,]\d+)?)\s*(?:-|–|A|TO)?\s*(\d+(?:[.,]\d+)?)?\s*BAR", t)
    if m: kv["presion_bar"] = f"{m.group(1)}-{m.group(2)}" if m.group(2) else m.group(1)

    # Potencia (HP o KW)
    m = re.search(r"(\d+(?:[.,]\d+)?)\s*HP", t)
    if m: kv["hp_motor"] = m.group(1)

    m = re.search(r"(\d+(?:[.,]\d+)?)\s*KW", t)
    if m: kv["kw_motor"] = m.group(1)

    # Viscosidad (cps)
    m = re.search(r"(\d+)\s*-\s*(\d+)\s*CPS", t)
    if m:
        kv["rango_cps_min"] = m.group(1)
        kv["rango_cps_max"] = m.group(2)

    # Bombas operativas + respaldo (ej. 1+1)
    m = re.search(r"\b(\d+)\s*\+\s*(\d+)\b", t)
    if m:
        kv["bombas_operativas"] = m.group(1)
        kv["bombas_respaldo"] = m.group(2)

    # Materiales
    if "304" in t: kv["material_tanque"] = "304SS"
    if "316" in t: kv["material_mojado"] = "316SS"

    # Norma API 675
    if "API 675" in t: kv["api_675"] = "true"

    return kv



In [ ]:
# --- Paso 2: extracción de contenido del PDF (robusto a páginas con tablas) --- 
import json, re, pdfplumber
from typing import List
from langchain_core.documents import Document  # ✅

# --- Heurística: detectar texto "corrupto/gibberish" típico de tablas mal leídas ---
def is_gibberish_text(t: str) -> bool:
    if not t or not t.strip():
        return True
    s = " ".join(t.split())

    # 1) Muchos puntos seguidos (líneas de puntos / leaders)
    if re.search(r"\.{5,}", s):
        return True

    # 2) Secuencias largas de letras sueltas separadas por espacios
    if re.search(r"(?:\b\w\b\s+){6,}", s):
        return True

    # 3) Ratio de tokens de una sola letra elevado
    tokens = re.findall(r"[A-Za-zÁÉÍÓÚÜÑáéíóúüñ]+", s)
    if tokens:
        single = sum(1 for tok in tokens if len(tok) == 1)
        if single / max(len(tokens), 1) > 0.35:
            return True

    return False

def extract_pdf_content(filepath: str) -> List[Document]:
    """
    Extrae texto y tablas con metadatos:
      - type: text | table
      - prio: hoja_datos | alcance | requisito | norma | general
      - kv: dict (serializado) cuando se detecta info técnica
    Reglas:
      • Si la página tiene tablas y el texto plano luce "gibberish", no indexar el texto.
      • Para tablas: convertimos a texto legible y combinamos KV por estructura y por regex.
    """
    docs: List[Document] = []

    with pdfplumber.open(filepath) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            # --- 1) Tablas primero (sabemos si hay tablas para decidir sobre texto) ---
            try:
                tables = page.extract_tables() or []
            except Exception:
                tables = []
            has_tables = bool(tables)

            table_docs_this_page: List[Document] = []
            for trows in tables:
                table_text, kv_table = table_to_text_and_kv(trows)
                table_text = clean_text(table_text)
                if not table_text:
                    continue

                prio = detect_priority(table_text)
                md = {"source": filepath, "page": i, "type": "table", "prio": prio}

                # KV por regex + por estructura de 2 columnas
                kv_detected = kv_from_text_page(table_text)
                if kv_table:
                    kv_detected = {**kv_detected, **kv_table}
                if kv_detected:
                    md["kv"] = json.dumps(kv_detected, ensure_ascii=False)

                table_docs_this_page.append(Document(page_content=table_text, metadata=md))

            # --- 2) Texto plano (solo si vale la pena) ---
            raw_text = page.extract_text() or ""
            clean = clean_text(raw_text)
            text_looks_bad = is_gibberish_text(clean)

            # Si hay tablas y el texto luce corrupto, NO añadimos el texto de la página
            add_text = bool(clean) and not (has_tables and text_looks_bad)

            if add_text:
                prio = detect_priority(clean)
                md = {"source": filepath, "page": i, "type": "text", "prio": prio}

                kv_text = kv_from_text_page(clean)
                if kv_text:
                    md["kv"] = json.dumps(kv_text, ensure_ascii=False)

                docs.append(Document(page_content=clean, metadata=md))

            # --- 3) Añadir las tablas (siempre que tengan contenido) ---
            docs.extend(table_docs_this_page)

    return docs


In [4]:
# ============================================
# Paso 3 - Indexar en Chroma (persistente)
# ============================================
from collections import Counter
from langchain_chroma import Chroma   # ✅ usar langchain_chroma
from langchain_openai import OpenAIEmbeddings

# Configuración global
VDB_PATH = "./vectordb"
COLLECTION_NAME = "autoselx_docs"

def reset_collection():
    """Elimina la colección persistente (para recargar desde cero)."""
    import shutil, os
    if os.path.exists(VDB_PATH):
        shutil.rmtree(VDB_PATH)
        print(f"🗑️ Colección eliminada en {VDB_PATH}")

def _safe_persist(vs):
    """Compatibilidad de persistencia entre versiones de Chroma/langchain."""
    try:
        if hasattr(vs, "persist"):
            vs.persist()  # versiones antiguas
        elif hasattr(vs, "_client") and hasattr(vs._client, "persist"):
            vs._client.persist()  # versiones nuevas
    except Exception as e:
        print(f"(i) No se pudo forzar persistencia explícita: {e}")

def add_files_to_vectordb(filepath: str, reset: bool = False):
    """
    1) Extrae documentos del PDF (texto + tablas + KV)
    2) Aplica chunking conservando metadatos
    3) Indexa en Chroma persistente
    4) Muestra diagnóstico resumido
    """
    if reset:
        reset_collection()

    # --- 1) Extraer documentos brutos ---
    raw_docs = extract_pdf_content(filepath)

    # --- 2) Chunking ---
    splits = chunk_documents(raw_docs)

    # --- 3) Indexar en Chroma persistente ---
    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=OpenAIEmbeddings(),
        persist_directory=VDB_PATH,
        collection_name=COLLECTION_NAME
    )

    # 🔑 Persistir en disco (seguro en múltiples versiones)
    _safe_persist(vectorstore)

    # --- 4) Diagnóstico ---
    by_type = Counter(d.metadata.get("type", "text") for d in splits)
    by_prio = Counter(d.metadata.get("prio", "general") for d in splits)
    total_kv = sum(1 for d in splits if "kv" in d.metadata)

    try:
        n_in_db = vectorstore._collection.count()
        print(f"📦 Documentos en DB (colección real): {n_in_db}")
    except Exception:
        print("(i) No se pudo obtener conteo directo desde DB")

    print(f"✅ {len(splits)} fragmentos indexados en {VDB_PATH} (colección '{COLLECTION_NAME}')")
    print("   📊 Por tipo:", dict(by_type))
    print("   📊 Por prioridad:", dict(by_prio))
    print(f"   🔑 Documentos con KV extraídos: {total_kv}")

    return vectorstore


In [5]:
# ============================================
# Paso 4 - Carga incremental y diagnóstico extendido
# ============================================
from collections import Counter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# === Utilidades de normalización y hashing  ===
import hashlib
import unicodedata
import re

def _normalize_for_hash(s: str) -> str:
    s = unicodedata.normalize("NFKC", s)
    s = s.lower().strip()
    s = re.sub(r"\s+", " ", s)
    return s

def make_doc_id(meta: dict, content: str) -> str:
    """ID estable basado en (source|page|type|contenido normalizado)."""
    key = f"{meta.get('source','')}|{meta.get('page','')}|{meta.get('type','')}|{_normalize_for_hash(content)}"
    return hashlib.sha1(key.encode("utf-8")).hexdigest()

def dedupe_by_hash(docs: List[Document]) -> tuple[list[Document], list[str], int]:
    """Elimina duplicados por contenido normalizado. Devuelve (docs_unicos, ids, n_drops)."""
    seen = set()
    out_docs, ids = [], []
    drops = 0
    for d in docs:
        h = make_doc_id(d.metadata, d.page_content)
        if h in seen:
            drops += 1
            continue
        seen.add(h)
        # Guarda el hash en metadatos (útil para debug)
        d.metadata["content_hash"] = h
        out_docs.append(d)
        ids.append(h)
    return out_docs, ids, drops



def add_files_to_vectordb(filepath: str, reset: bool = False):
    if reset:
        reset_collection()

    # 1) Extraer
    raw_docs = extract_pdf_content(filepath)

    # 2) Chunking
    splits = chunk_documents(raw_docs)

    # 2.1) Dedupe por contenido
    unique_docs, ids, dropped = dedupe_by_hash(splits)

    # 3) Indexar con IDs deterministas
    vectorstore = Chroma.from_documents(
        documents=unique_docs,
        embedding=OpenAIEmbeddings(),
        persist_directory=VDB_PATH,
        collection_name=COLLECTION_NAME,
        ids=ids,  # 👈 clave
    )

    _safe_persist(vectorstore)

    # 4) Diagnóstico
    by_type = Counter(d.metadata.get("type", "text") for d in unique_docs)
    by_prio = Counter(d.metadata.get("prio", "general") for d in unique_docs)
    total_kv = sum(1 for d in unique_docs if "kv" in d.metadata)

    try:
        n_in_db = vectorstore._collection.count()
        print(f"📦 Documentos en DB (colección real): {n_in_db}")
    except Exception:
        n_in_db = None

    print(f"✅ {len(unique_docs)} fragmentos (tras dedupe; {dropped} descartados) indexados en {VDB_PATH} (colección '{COLLECTION_NAME}')")
    print("   📊 Por tipo:", dict(by_type))
    print("   📊 Por prioridad:", dict(by_prio))
    print(f"   🔑 Documentos con KV extraídos: {total_kv}")

    return vectorstore





In [6]:
# ============================================
# Paso 5 - Utilidad para inspeccionar colección
# ============================================
from collections import Counter
import chromadb

def get_collection_diagnostics(show_samples: int = 0):
    """
    Inspecciona la colección persistente y devuelve:
    - Fuentes únicas
    - Distribución por prioridad, tipo, páginas
    - Conteo de documentos con KV
    Opcional: muestra N ejemplos de metadatos
    """
    client = chromadb.PersistentClient(path=VDB_PATH)
    collection = client.get_collection(name=COLLECTION_NAME)

    data = collection.get(include=["documents", "metadatas"])
    metadatas = data.get("metadatas", [])
    documents = data.get("documents", [])

    sources = [m.get("source", "unknown").split("/")[-1] for m in metadatas if m]
    prio_counts = Counter(m.get("prio", "general") for m in metadatas if m)
    type_counts = Counter(m.get("type", "text") for m in metadatas if m)
    page_counts = Counter(m.get("page", "na") for m in metadatas if m)
    kv_total = sum(1 for m in metadatas if m and "kv" in m)

    print(f"📦 Colección: {COLLECTION_NAME}")
    print(f"📁 Fuentes únicas: {len(set(sources))} → {sorted(set(sources))}")
    print("   📊 Distribución por prioridad:", dict(prio_counts))
    print("   📊 Distribución por tipo:", dict(type_counts))
    print("   📊 Páginas únicas indexadas:", len(page_counts))
    print(f"   🔑 Documentos con KV: {kv_total}")

    if show_samples > 0:
        print("\n🔍 Ejemplos de metadatos:")
        for i, m in enumerate(metadatas[:show_samples], start=1):
            print(f"{i}.", m)

    return {
        "sources": sorted(set(sources)),
        "prio_counts": dict(prio_counts),
        "type_counts": dict(type_counts),
        "page_counts": dict(page_counts),
        "kv_total": kv_total,
        "total_docs": len(documents),
    }

# Ejemplo:
# resumen = get_collection_diagnostics(show_samples=3)
# print(resumen)


In [7]:
# ============================================
# Paso 5.1 - Chunking con preservación de metadatos
# ============================================
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document  # ✅ Import actualizado

def chunk_documents(
    docs: List[Document],
    chunk_size: int = 800,
    chunk_overlap: int = 100,
    verbose: bool = False
) -> List[Document]:
    """
    Divide documentos largos en chunks manejables para embeddings,
    conservando metadatos (source, page, type, prio, kv).
    
    - chunk_size: tamaño máximo de cada fragmento
    - chunk_overlap: solapamiento entre chunks
    - verbose: si True, imprime resumen por documento
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )

    chunks: List[Document] = []

    for doc in docs:
        splits = splitter.split_text(doc.page_content)
        valid_splits = [s for s in splits if s.strip()]  # evitar basura vacía

        for idx, chunk in enumerate(valid_splits, start=1):
            new_meta = doc.metadata.copy()
            new_meta["chunk_id"] = f"{new_meta.get('page','na')}_{idx}"

            # Mantener prio y kv si existen
            if "prio" not in new_meta:
                new_meta["prio"] = "general"
            if "kv" in doc.metadata:
                new_meta["kv"] = doc.metadata["kv"]

            chunks.append(Document(page_content=chunk, metadata=new_meta))

        if verbose:
            print(f"📄 p.{doc.metadata.get('page','?')} "
                  f"| {doc.metadata.get('type','text')} "
                  f"→ {len(valid_splits)} chunks")

    if verbose:
        print(f"\n✅ Total de chunks generados: {len(chunks)}")

    return chunks



In [8]:

# --- Paso 6: correr el pipeline ---
if __name__ == "__main__":
    # 🔄 Limpia todo si quieres partir desde cero:
    # reset_collection()

    vectorstore = add_files_to_vectordb(
        "./data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf",
        reset=True  # pon True si quieres borrar la colección antes
    )

    print("\n🚀 Pipeline completado. La colección está lista para consultas.")


🗑️ Colección eliminada en ./vectordb
📦 Documentos en DB (colección real): 596
✅ 596 fragmentos (tras dedupe; 0 descartados) indexados en ./vectordb (colección 'autoselx_docs')
   📊 Por tipo: {'text': 356, 'table': 240}
   📊 Por prioridad: {'alcance': 80, 'norma': 82, 'general': 126, 'hoja_datos': 203, 'requisito': 105}
   🔑 Documentos con KV extraídos: 167

🚀 Pipeline completado. La colección está lista para consultas.


In [9]:
# ============================================
# Verificar qué documentos hay en vectordb
# ============================================
import chromadb

# Conectar al cliente persistente en la ruta que estás usando
PERSIST_DIR = "/home/user/Desktop/Tesis2025/AutoSelectX/scriptsSampleRAG/vectordb"
client = chromadb.PersistentClient(path=PERSIST_DIR)

# Nombre de la colección que usaste al crear los embeddings
COLLECTION_NAME = "autoselx_docs"

def get_unique_sources_list(client, collection_name):
    try:
        collection = client.get_collection(collection_name)
        # Obtener todos los documentos y metadatos
        data = collection.get(include=["metadatas"])
        metadatas = data.get("metadatas", [])
        
        sources = set()
        for metadata in metadatas:
            if metadata and "source" in metadata:
                sources.add(metadata["source"])
        
        # Extraer solo el nombre de archivo
        file_names = sorted(set(source.split("/")[-1] for source in sources))
        return file_names
    except Exception as e:
        return f"⚠️ Error al acceder a la colección: {e}"

# Ejecutar
docs_in_vdb = get_unique_sources_list(client, COLLECTION_NAME)
print("📂 Documentos en vectordb:", docs_in_vdb)



📂 Documentos en vectordb: ['CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf']


In [10]:
# ============================================
# Paso 7 - Debug robusto: explorar fragmentos (MMR con fallback, mejora alcance) TEST!!!!!!!!!!!!!!!!!!!!!!!
# ============================================
import json

def _try_retrieve(query: str, search_kwargs: dict, use_mmr: bool = True):
    """Envuelve as_retriever con fallback y logs amigables."""
    try:
        retriever = vectorstore.as_retriever(
            search_type="mmr" if use_mmr else "similarity",
            search_kwargs=search_kwargs,
        )
        return retriever.get_relevant_documents(query)
    except Exception as e:
        modo = "MMR" if use_mmr else "similarity"
        print(f"(i) Falló {modo} con error {e.__class__.__name__}: {e}")
        return []

def debug_busqueda_alcance(
    q: str = "¿Cuál es el alcance del suministro?",
    k_text: int = 5,
    k_mixed: int = 5,
    preferir_texto: bool = True,
):
    print(f"🔎 Consulta: {q}")

    # --- 0) Chequeo rápido de colección
    try:
        coll = vectorstore._collection
        _probe = coll.get(limit=3)
        if not _probe.get("ids"):
            print("(i) Colección vacía o corrupta; considera resetear y reindexar.")
    except Exception as e:
        print(f"(i) No pude inspeccionar la colección: {e}")

    # --- 1) Fase A: priorizar TEXTO con prio ∈ {alcance, general, hoja_datos}
    kwargs_a = {
        "k": max(k_text, 5),
        "fetch_k": 40,
        "lambda_mult": 0.4,
        "filter": {"prio": {"$in": ["alcance", "general", "hoja_datos"]}},  # 👈 ampliado
    }

    # Intento MMR
    res_text = _try_retrieve(q, kwargs_a, use_mmr=True)
    # Fallback similarity
    if not res_text:
        kwargs_a_fallback = {
            "k": max(k_text, 5),
            "filter": {"prio": {"$in": ["alcance", "general", "hoja_datos"]}},
        }
        res_text = _try_retrieve(q, kwargs_a_fallback, use_mmr=False)

    # Post-filtro por tipo en Python
    if preferir_texto:
        res_text = [r for r in res_text if r.metadata.get("type") == "text"]

    print(f"\n📚 TEXT preferido (alcance/general/hoja_datos) → {len(res_text)}")

    # --- Mostrar únicos
    seen_hash = set()
    def _mark_and_show(prefix: str, doc, idx: int):
        ch = doc.metadata.get("content_hash") or f"{doc.metadata.get('page')}_{doc.metadata.get('chunk_id')}"
        if ch in seen_hash:
            return False
        seen_hash.add(ch)
        print(f"\n--- {prefix} {idx} ---")
        print((doc.page_content[:400]).replace("\n", " "))
        print("Metadatos:", {k: doc.metadata.get(k) for k in ["page","type","prio","source","chunk_id"]})
        if "kv" in doc.metadata:
            try:
                print("KV:", json.loads(doc.metadata["kv"]))
            except Exception:
                print("KV (raw):", doc.metadata["kv"])
        return True

    shown = 0
    for i, r in enumerate(res_text, 1):
        if _mark_and_show("TEXT", r, i):
            shown += 1

    # --- 2) Fase B: mezcla (text+table) como respaldo
    kwargs_b = {"k": max(k_mixed, 5)}
    res_mixed = _try_retrieve(q, kwargs_b, use_mmr=False)  # similarity estable
    print(f"\n🧩 MIX (text+table) → {len(res_mixed)}")

    added = 0
    for r in res_mixed:
        if _mark_and_show("MIX", r, added + 1):
            added += 1
        if added >= k_mixed:
            break

    print(f"\n📌 Total únicos mostrados: {len(seen_hash)}")


# 🚀 Ejecutar debug
debug_busqueda_alcance()


🔎 Consulta: ¿Cuál es el alcance del suministro?


/tmp/ipykernel_21937/2087002342.py:13: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return retriever.get_relevant_documents(query)



📚 TEXT preferido (alcance/general/hoja_datos) → 3

--- TEXT 1 ---
2.0 ALCANCE Este documento describe el alcance del suministro y servicios del PROVEEDOR, sus responsabilidades, plan de ejecución y demás aspectos generales para el diseño; fabricación; suministro; asistencia para el montaje y pruebas del PAQUETE de inyección de químicos de Polímero Aniónico AX-75317 y los Quill´s de Inyección Retráctil roscado, completamente operacionales y funcionales. Los detal
Metadatos: {'page': 58, 'type': 'text', 'prio': 'hoja_datos', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'chunk_id': '58_2'}

--- TEXT 2 ---
la instalación de los PAQUETES, teniendo en cuenta el caso que se requiera transporte terrestre, fluvial y/o aéreo, y debe ser incluido el plan general de movilización en la respectiva oferta, y confirmado detalladamente después de puesta la orden de compra. De esta manera el CLIENTE para la fecha de llegada podrá contar con las bases y equipos necesari

In [11]:
# ==========================
# Complemento Paso 7 - Función auxiliar de debug
# ==========================
def debug_query(vectorstore, query, prio=None, k=5):
    """
    Ejecuta una búsqueda con filtro de metadatos (cuando prio != None).
    Incluye fallbacks para prioridades relacionadas y muestra resultados
    con metadatos enriquecidos y KV cuando existen.
    """
    import json

    # Prios relacionadas para fallback (más tolerancia)
    prio_fallbacks = {
        "alcance":    ["alcance", "general", "hoja_datos"],
        "requisito":  ["requisito", "hoja_datos", "general"],
        "norma":      ["norma", "general"],
        "hoja_datos": ["hoja_datos", "requisito", "general"],
        "general":    ["general", "alcance", "hoja_datos"]
    }

    print(f"\n🔎 DEBUG QUERY: '{query}' (prio={prio})\n")

    def run_search(allowed_prios, topk):
        if allowed_prios is None:
            # búsqueda sin filtro
            return vectorstore.similarity_search(query, k=topk)
        else:
            # búsqueda con filtro por metadatos
            return vectorstore.similarity_search(
                query,
                k=topk,
                filter={"prio": {"$in": allowed_prios}}
            )

    # --- 1) Intento principal con filtro si prio está definido
    if prio:
        allowed = prio_fallbacks.get(prio, [prio])
        docs = run_search(allowed_prios=allowed, topk=max(k, 10))

        # --- 2) Fallback si no hay resultados
        if not docs:
            allowed_wide = list(set(allowed + ["general"]))
            docs = run_search(allowed_prios=allowed_wide, topk=50)

        if not docs:
            print("⚠️ No se encontraron resultados con esa prioridad (ni con fallback).")
            return
    else:
        # búsqueda sin restricción de prioridad
        docs = run_search(allowed_prios=None, topk=k)

    # --- Mostrar resultados
    print(f"📊 Fragmentos recuperados: {len(docs)}\n")
    seen = set()
    for i, r in enumerate(docs, 1):
        ch = r.metadata.get("content_hash") or f"{r.metadata.get('page')}_{r.metadata.get('chunk_id')}"
        if ch in seen:
            continue
        seen.add(ch)

        print(f"--- CHUNK {i} ---")
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", {k: r.metadata.get(k) for k in ["page","type","prio","source","chunk_id"]})
        if "kv" in r.metadata:
            try:
                print("KV dict:", json.loads(r.metadata["kv"]))
            except Exception:
                print("KV (raw):", r.metadata["kv"])
        print()

    print(f"📌 Total únicos mostrados: {len(seen)}")


In [12]:
# ==========================
# Pruebas de debug por prioridad
# ==========================

# Alcance del suministro
debug_query(vectorstore, "alcance del suministro", prio="alcance")


🔎 DEBUG QUERY: 'alcance del suministro' (prio=alcance)

📊 Fragmentos recuperados: 10

--- CHUNK 1 ---
SECCIÓN | DESCRIPCIÓN 2 | ALCANCE DEL SUMINISTRO 10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)
Metadatos: {'page': 1, 'type': 'table', 'prio': 'alcance', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'chunk_id': '1_1'}
KV dict: {'SECCIÓN': 'DESCRIPCIÓN', '2': 'ALCANCE DEL SUMINISTRO', '10.3': 'DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)'}

--- CHUNK 2 ---
2.0 ALCANCE Este documento describe el alcance del suministro y servicios del PROVEEDOR, sus responsabilidades, plan de ejecución y demás aspectos generales para el diseño; fabricación; suministro; asistencia para el montaje y pruebas del PAQUETE de inyección de químicos de Polímero Aniónico AX-75317 y los Quill´s de Inyección Retráctil roscado, completamente operacionales y funcionales. Los detal
Metadatos: {'page': 58, 'type': 'text', 'prio': 'hoja_datos', 'source': './data/CA

In [13]:
debug_query(vectorstore, "API 675 caudal presión material", prio="hoja_datos")


🔎 DEBUG QUERY: 'API 675 caudal presión material' (prio=hoja_datos)

📊 Fragmentos recuperados: 10

--- CHUNK 1 ---
UNIDAD COMPLETA      CERTIFICADOS DE MATERIAL APLICADO A UNIDAD PAQUETE OTROS: API 675 BOMBAS DE DESPLAZAMIENTO POSITIVO - VOL. CONTROLADO  OTROS: | | | | | | | | | | | | | | | | | | | |   | PREPARACIÓN DE ENVÍO | | | | | | | | | | | | | | | | | | | |   | PREPARACIÓN DOMÉSTICA:  POR PROVEEDOR  POR COMPRADOR  OBSERVACIONES CAJA DE EXPORTACIÓN:  POR PROVEEDOR  POR COMPRADOR  OBSERVACIONE
Metadatos: {'page': 42, 'type': 'table', 'prio': 'hoja_datos', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'chunk_id': '42_11'}
KV dict: {'caudal_gph': '12-18', 'presion_psig': '40', 'hp_motor': '1', 'kw_motor': '5', 'rango_cps_min': '500', 'rango_cps_max': '1000', 'bombas_operativas': '1', 'bombas_respaldo': '1', 'material_tanque': '304SS', 'material_mojado': '316SS', 'api_675': 'true'}

--- CHUNK 2 ---
VICEPRESIDENCIA DE INGENIERÍA Y PROYECTOS

In [14]:
debug_query(vectorstore, "RETIE y normas aplicables", prio="norma")


🔎 DEBUG QUERY: 'RETIE y normas aplicables' (prio=norma)

📊 Fragmentos recuperados: 10

--- CHUNK 1 ---
No es aplicable Precauciones particulares para los usuarios Ninguno/a 15. Información reglamentaria Los acuerdos internacionales Protocolo de Montreal - Sustancias Agotadoras del Ozono: No aplica Convención Estocolmo - Contaminantes Orgánicos Persistentes: No aplica Convenio de Rótterdam - Consentimiento Fundamentado Previo: No aplica Convenio de Basilea - Residuos Peligrosos: No aplica Calificaci
Metadatos: {'page': 52, 'type': 'text', 'prio': 'norma', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'chunk_id': '52_3'}

--- CHUNK 2 ---
19 V R PINTURA/ PROTECCIÓN EXTERIOR / INTERIOR 20 V R INSPECCIÓN FINAL 21 R DOSSIER OPERACIÓN / INSTALACIÓN Y MANTENIMIENTO DEL PROVEEDOR 22 V R DOSSIER/ DISEÑO/ FABRICACIÓN 23 R PREPARACIÓN PARA EMBARQUE 24 V R NOTAS DE TERMINACIÓN (VER NOTA 2) GLOSARIO A/Atestiguar PUNTO DE ESPERA del COMPRADOR, el PROVEEDOR/FABRICANTE

In [15]:
debug_query(vectorstore, "requisitos técnicos del sistema de dosificación", prio="requisito")


🔎 DEBUG QUERY: 'requisitos técnicos del sistema de dosificación' (prio=requisito)

📊 Fragmentos recuperados: 10

--- CHUNK 1 ---
de acuerdo con la hoja de datos Paquete De Química STAP EC3 CAS-09991-MER-HD-000003 a fin de garantizar una unidad funcional completa. El PROVEEDOR debe cumplir con los requerimientos mínimos requeridos por el cliente para entregar un sistema de dosificación efectivo y que cumpla con las expectativas en la operación y producción de la estación. EDP-F-046 – 20/11/2017 V-1 8/36 Documento Rector: EDP
Metadatos: {'page': 8, 'type': 'text', 'prio': 'hoja_datos', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'chunk_id': '8_5'}

--- CHUNK 2 ---
95 96 97 98 | 2 | Se requiere una unidad paquete de Preparación y Dosificación sobre patín, conformada al menos por: Un tanque horizontal de 1056 galones de acero inoxidable 304ss con dos cámaras de preparación, y una camara para dosificación. Este paquete también incluye un sistema de contro

In [16]:
# ============================================
# Paso 8 - Probar extracción de Hoja de Datos API 675
# ============================================
import json

def buscar_hojas_datos(pregunta: str = "hoja de datos bomba", k: int = 12):
    # --- 1) Buscamos primero chunks con prio=hoja_datos ---
    retriever = vectorstore.as_retriever(search_kwargs={"k": k})
    results = retriever.get_relevant_documents(pregunta)
    hd_chunks = [r for r in results if r.metadata.get("prio") == "hoja_datos"]

    # --- 2) Si hay pocos resultados, ampliamos con los que tengan KV ---
    if len(hd_chunks) < 3:
        kv_chunks = [r for r in results if "kv" in r.metadata]
        filtrados = hd_chunks + kv_chunks
    else:
        filtrados = hd_chunks

    print(f"🔎 Total de chunks candidatos (HD/kv): {len(filtrados)}")

    count = 0
    for r in filtrados:
        has_kv = "kv" in r.metadata
        if has_kv:
            count += 1

        flag = "🔑" if has_kv else ""
        print(f"\n--- CHUNK HD {flag} ---")
        print(f"p.{r.metadata.get('page')} | {r.metadata.get('type')} | prio={r.metadata.get('prio')}")
        print(r.page_content[:400])

        if has_kv:
            try:
                kv = json.loads(r.metadata["kv"])
                print("KV:", kv)
            except Exception:
                print("KV (raw):", r.metadata["kv"])

    print(f"✅ Chunks con KV reales: {count}")
    return filtrados


# Ejemplo:
buscar_hojas_datos("Datos de diseño de bomba dosificadora API 675 (caudal, presión, HP)")



🔎 Total de chunks candidatos (HD/kv): 10

--- CHUNK HD 🔑 ---
p.43 | text | prio=hoja_datos
96 2 agua, para la introducción del polímero sólido, agitacion, alarma para vaciado y llenado. para la dosificacion debe contar con Una (1) bomba dosificadora de diafragma para la inyección de polímero líquido preparado de 1Hp,12 - 18 gph @ 100psi, API 675, con una (1) de respaldo accionadas por motores eléctricos. El
paquete contara con un sistema de control PLC dedicado a gestionar todas las fun
KV: {'caudal_gph': '12-18', 'presion_psig': '100', 'hp_motor': '1', 'material_tanque': '304SS', 'api_675': 'true'}

--- CHUNK HD  ---
p.37 | text | prio=hoja_datos
22 Clasificación del área NO Area No Clasificada
23 Equipo apto para Intemperie? SI SI
24 Fluido de operación SI Agua Filtrada
25 Temperatura de bombeo T.B. (°F) NO -
26 Densidad relativa @ T.B. NO -
27 Viscosidad @ T.B. (cP) SI 500-1000
28 Presión de vapor @ T.B. (Psia) NO -
29 Presión de succión (psig) NO -
30 Presión de descarga (psig) SI 

[Document(id='e36e41d49df836bcc9550de6288029b90298eee9', metadata={'kv': '{"caudal_gph": "12-18", "presion_psig": "100", "hp_motor": "1", "material_tanque": "304SS", "api_675": "true"}', 'type': 'text', 'prio': 'hoja_datos', 'chunk_id': '43_3', 'page': 43, 'content_hash': 'e36e41d49df836bcc9550de6288029b90298eee9', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf'}, page_content='96 2 agua, para la introducción del polímero sólido, agitacion, alarma para vaciado y llenado. para la dosificacion debe contar con Una (1) bomba dosificadora de diafragma para la inyección de polímero líquido preparado de 1Hp,12 - 18 gph @ 100psi, API 675, con una (1) de respaldo accionadas por motores eléctricos. El\npaquete contara con un sistema de control PLC dedicado a gestionar todas las funciones del paquete, sus cajas (Potencia, Instrumentación, Instrumentos y Control NEMA 4X (Área no clasificada). La instrumentación mínima asociada al paquete está indicada en el P&ID CAS-0

In [17]:
# ============================================
# Paso 8.1 - Consolidar Hoja de Datos API 675
# ============================================
import json
from typing import Dict, Any


def extraer_campos_extras(texto: str) -> Dict[str, str]:
    """Regex para campos no capturados aún en kv_from_text_page"""
    t = texto.upper()
    kv = {}

    # Fluido de operación
    m = re.search(r"FLUIDO\s+DE\s+OPERACI[ÓO]N\s+SI\s+([A-ZÁÉÍÓÚ0-9\s]+)", t)
    if m:
        kv["fluido"] = m.group(1).title()

    # Viscosidad cP
    m = re.search(r"VISCO(SIDAD)?[^0-9]*(\d{2,5})\s*-\s*(\d{2,5})\s*CP", t)
    if m:
        kv["viscosidad_cps"] = f"{m.group(2)}-{m.group(3)}"

    # Eficiencia %
    m = re.search(r"EFICIENCIA.*?(\d{1,3})\s*%|≥\s*(\d{1,3})", t)
    if m:
        kv["eficiencia_pct"] = m.group(1) or m.group(2)

    # Clasificación de área
    m = re.search(r"CLASIFICACI[ÓO]N\s+DEL\s+ÁREA\s+NO\s+([A-Z\s]+)", t)
    if m:
        kv["clasificacion_area"] = m.group(1).title()

    return kv

def consolidar_hoja_datos(pregunta="hoja de datos bomba API 675", k=12) -> Dict[str, Any]:
    docs = buscar_hojas_datos(pregunta, k=k)
    consolidado = {}

    for d in docs:
        # 1) KV en metadata
        if "kv" in d.metadata:
            try:
                kv = json.loads(d.metadata["kv"])
                consolidado.update(kv)
            except Exception:
                pass

        # 2) Extra campos con regex sobre el contenido
        extras = extraer_campos_extras(d.page_content)
        consolidado.update(extras)

    print("\n✅ JSON consolidado:")
    print(json.dumps(consolidado, indent=2, ensure_ascii=False))
    return consolidado

# Ejemplo:
consolidar_hoja_datos("Datos de diseño de bomba dosificadora API 675")


🔎 Total de chunks candidatos (HD/kv): 10

--- CHUNK HD 🔑 ---
p.43 | text | prio=hoja_datos
96 2 agua, para la introducción del polímero sólido, agitacion, alarma para vaciado y llenado. para la dosificacion debe contar con Una (1) bomba dosificadora de diafragma para la inyección de polímero líquido preparado de 1Hp,12 - 18 gph @ 100psi, API 675, con una (1) de respaldo accionadas por motores eléctricos. El
paquete contara con un sistema de control PLC dedicado a gestionar todas las fun
KV: {'caudal_gph': '12-18', 'presion_psig': '100', 'hp_motor': '1', 'material_tanque': '304SS', 'api_675': 'true'}

--- CHUNK HD 🔑 ---
p.43 | text | prio=hoja_datos
CAS-09991-MER-HD-000003
INGENIERÍA DETALLADA ESTACIONES MÓDULO INTEGRAL
Documento No.
HOJAS DE DATOS PAQUETE DE QUÍMICO STAP EC3 Revisión:AFC4
82 PROPUESTA PESOS Y DIMCOENMSPIROANES (NOTA 9) AS BUIT
83 DIMENSIONES
DESCRIPCIÓN MODELO CANTIDAD PAT Í (N co D n E p rP uR ebE aP hA idR rA osC tI áÓ ticN a ( )lbs) BOMBAS DE DOSIFICACIÓN (lbs) TOTA

{'caudal_gph': '12-18',
 'presion_psig': '100',
 'hp_motor': '1',
 'material_tanque': '304SS',
 'api_675': 'true',
 'kw_motor': '5',
 'bombas_operativas': '1',
 'bombas_respaldo': '1',
 'material_mojado': '316SS',
 'rango_cps_min': '500',
 'rango_cps_max': '1000'}

In [18]:
# ============================================
# Paso 9 - Inspeccionar metadatos del Alcance
# ============================================

def buscar_alcance(pregunta: str = "alcance del suministro", k: int = 8):
    """Busca chunks relacionados con alcance (prio=alcance) y muestra metadatos"""
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": k, "filter": {"prio": "alcance"}}
    )
    results = retriever.get_relevant_documents(pregunta)

    print(f"🔎 Total de chunks recuperados (prio=alcance): {len(results)}")
    for r in results:
        print(
            f"\n--- CHUNK p.{r.metadata.get('page')} | "
            f"{r.metadata.get('type')} | prio={r.metadata.get('prio')} ---"
        )
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)

    return results

# Ejemplo de prueba:
buscar_alcance()



🔎 Total de chunks recuperados (prio=alcance): 8

--- CHUNK p.1 | table | prio=alcance ---
SECCIÓN | DESCRIPCIÓN 2 | ALCANCE DEL SUMINISTRO 10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)
Metadatos: {'page': 1, 'kv': '{"SECCIÓN": "DESCRIPCIÓN", "2": "ALCANCE DEL SUMINISTRO", "10.3": "DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)"}', 'type': 'table', 'prio': 'alcance', 'chunk_id': '1_1', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'content_hash': 'ba1abd605893ad92eede20c951e0fb6d45d22a3b'}

--- CHUNK p.73 | text | prio=alcance ---
del fluido, así como garantizar, verificar y asegurar los cálculos y diseños detallados de las cimentaciones para todos los equipos y componentes alcance de su suministro (incluye soportes). CAS09991MERET000002_InyeccionQxSTAPEC3_AFC2.doc 17 de 18
Metadatos: {'content_hash': '20dab9928a81607b6c21f59b71a3406dfb85daf8', 'chunk_id': '73_6', 'type': 'text', 'page': 73, 'source': './data/CAS09991MERMR000003_Inyec

[Document(id='ba1abd605893ad92eede20c951e0fb6d45d22a3b', metadata={'page': 1, 'kv': '{"SECCIÓN": "DESCRIPCIÓN", "2": "ALCANCE DEL SUMINISTRO", "10.3": "DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)"}', 'type': 'table', 'prio': 'alcance', 'chunk_id': '1_1', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'content_hash': 'ba1abd605893ad92eede20c951e0fb6d45d22a3b'}, page_content='SECCIÓN | DESCRIPCIÓN\n2 | ALCANCE DEL SUMINISTRO\n10.3 | DOCUMENTOS DE REFERENCIA (ACTUALIZA ÚLTIMA REVISIÓN)'),
 Document(id='20dab9928a81607b6c21f59b71a3406dfb85daf8', metadata={'content_hash': '20dab9928a81607b6c21f59b71a3406dfb85daf8', 'chunk_id': '73_6', 'type': 'text', 'page': 73, 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'prio': 'alcance'}, page_content='del fluido, así como garantizar, verificar y asegurar los cálculos y diseños detallados de las\ncimentaciones para todos los equipos y componentes alcance de su suministro (incluye

In [19]:
# ============================================
# Paso 10 - Validar TODOS los metadatos (corregido)
# ============================================
from collections import Counter

def validar_metadatos_globales():
    """Valida consistencia de metadatos en la colección completa."""
    collection = vectorstore._collection
    data = collection.get(include=["metadatas", "documents"])

    total_docs = len(data["documents"])
    print(f"🔎 Total de registros en la colección: {total_docs}")

    errores = 0
    prio_counts = Counter()
    type_counts = Counter()
    kv_count = 0
    seen_ids = set()
    duplicados = 0

    for i, meta in enumerate(data["metadatas"]):
        doc_id = data["ids"][i]  # aquí sí podemos usarlo, ya viene siempre

        # Detectar duplicados de ID
        if doc_id in seen_ids:
            duplicados += 1
        else:
            seen_ids.add(doc_id)

        # Validar claves mínimas
        if not all(key in meta for key in ["source", "page", "type"]):
            print(f"⚠️ Registro {i} con metadatos incompletos:", meta)
            errores += 1
            continue

        # Validar valores
        if not isinstance(meta.get("page"), int) or meta["page"] <= 0:
            print(f"⚠️ Registro {i} con página inválida:", meta)
            errores += 1

        if not meta.get("type"):
            print(f"⚠️ Registro {i} con tipo vacío:", meta)
            errores += 1

        # Contadores útiles
        prio_counts[meta.get("prio", "none")] += 1
        type_counts[meta.get("type", "unknown")] += 1
        if "kv" in meta:
            kv_count += 1

    # Resumen
    if errores == 0:
        print("✅ Todos los documentos tienen metadatos completos y válidos.")
    else:
        print(f"⚠️ Se encontraron {errores} problemas de metadatos.")

    if duplicados > 0:
        print(f"⚠️ Se detectaron {duplicados} duplicados de IDs.")

    print("\n📊 Distribución por prioridad:", dict(prio_counts))
    print("📊 Distribución por tipo:", dict(type_counts))
    print(f"📊 Total de documentos con KV (tablas clave-valor): {kv_count}")

# Ejemplo:
validar_metadatos_globales()



🔎 Total de registros en la colección: 596
✅ Todos los documentos tienen metadatos completos y válidos.

📊 Distribución por prioridad: {'alcance': 80, 'norma': 82, 'general': 126, 'hoja_datos': 203, 'requisito': 105}
📊 Distribución por tipo: {'text': 356, 'table': 240}
📊 Total de documentos con KV (tablas clave-valor): 167


In [20]:
# ============================================
# Paso 11 - Buscar solo tablas o solo texto
# ============================================
def buscar_por_tipo(tipo: str = "table", pregunta: str = "bomba dosificadora", k: int = 5):
    """
    Recupera fragmentos filtrando por tipo ('table' o 'text').
    """
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": k, "filter": {"type": tipo}}
    )
    results = retriever.get_relevant_documents(pregunta)

    print(f"\n🔎 Resultados filtrados por tipo='{tipo}' | pregunta='{pregunta}' → {len(results)} encontrados")
    for i, r in enumerate(results, 1):
        print(f"\n--- CHUNK {i} ---")
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)

        if "kv" in r.metadata:  # Mostrar KV si existe
            try:
                import json
                print("KV:", json.loads(r.metadata["kv"]))
            except Exception:
                print("KV (raw):", r.metadata["kv"])

    return results


# Ejemplos de prueba:
buscar_por_tipo("table", "tags y cantidades")
buscar_por_tipo("text", "alcance del suministro")




🔎 Resultados filtrados por tipo='table' | pregunta='tags y cantidades' → 5 encontrados

--- CHUNK 1 ---
b. Cantidad por equipo según recomendado por el fabricante. c. Número de parte y catálogo del repuesto (drawing).
Metadatos: {'prio': 'norma', 'chunk_id': '19_2', 'page': 19, 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'content_hash': '379f10a10a3758d13d168c1f75220be7fcdae2d9', 'type': 'table'}

--- CHUNK 2 ---
Ítem | Cantidad | Unidad | TAG | Descripción | Código de Catálogo ECOPETROL 1 | 1 | UND | AX- 75317 | Patín de Preparación y Dosificación: Se requiere una unidad paquete de Preparación y Dosificación sobre patín, conformada al menos por: Un tanque horizontal de 1056 galones de acero inoxidable 304ss con dos cámaras de preparación, y una cámara para dosificación. Este paquete también incluye un sis
Metadatos: {'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'chunk_id': '6_1', 'page': 6, 'type': 'table', 'prio': 

[Document(id='72244995336010c9b13b13c4c30297b33132770a', metadata={'type': 'text', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'prio': 'hoja_datos', 'content_hash': '72244995336010c9b13b13c4c30297b33132770a', 'chunk_id': '58_2', 'page': 58}, page_content='2.0 ALCANCE\nEste documento describe el alcance del suministro y servicios del PROVEEDOR, sus\nresponsabilidades, plan de ejecución y demás aspectos generales para el diseño; fabricación;\nsuministro; asistencia para el montaje y pruebas del PAQUETE de inyección de químicos de Polímero\nAniónico AX-75317 y los Quill´s de Inyección Retráctil roscado, completamente operacionales y\nfuncionales.\nLos detalles relevantes a los requerimientos de proceso, características constructivas, garantías de\ndesempeño y otros requerimientos están incluidos en este documento.\nEsta especificación debe ser leída en conjunto con:\n• CAS-09991-MER-HD-000003: HOJA DE DATOS PAQUETE DE QUÍMICA STAP EC3.\n• CAS-09991-PRO-P

In [21]:
# ============================================
# Paso 11.1 - Comparar resultados table vs text
# ============================================
from collections import Counter
import json

def comparar_table_vs_text(pregunta="alcance del suministro", k=6):
    resultados = {}
    for tipo in ["table", "text"]:
        retriever = vectorstore.as_retriever(
            search_kwargs={"k": k, "filter": {"type": tipo}}
        )
        docs = retriever.get_relevant_documents(pregunta)
        resultados[tipo] = docs

        print(f"\n🔎 Resultados filtrados por tipo='{tipo}' → {len(docs)} encontrados")
        prio_counter = Counter([d.metadata.get("prio", "none") for d in docs])
        kv_count = sum(1 for d in docs if "kv" in d.metadata)

        print(f"📊 Distribución de prioridades: {dict(prio_counter)}")
        print(f"📊 Con KV extraídos: {kv_count}")

        for r in docs:
            print("\n--- CHUNK ---")
            print(r.page_content[:300].replace("\n", " "))
            print("Metadatos:", r.metadata)
            if "kv" in r.metadata:
                try:
                    print("KV:", json.loads(r.metadata["kv"]))
                except:
                    print("KV (raw):", r.metadata["kv"])

    # Comparación final
    print("\n================= RESUMEN COMPARATIVO =================")
    for tipo in ["table", "text"]:
        print(f"Tipo: {tipo} | Total: {len(resultados[tipo])}")
        kv_total = sum(1 for d in resultados[tipo] if "kv" in d.metadata)
        print(f" → Con KV: {kv_total}")
        prio_counter = Counter([d.metadata.get("prio", "none") for d in resultados[tipo]])
        print(f" → Prioridades: {dict(prio_counter)}")

# Ejemplo de prueba
comparar_table_vs_text("tags y cantidades", k=5)
comparar_table_vs_text("alcance del suministro", k=5)



🔎 Resultados filtrados por tipo='table' → 5 encontrados
📊 Distribución de prioridades: {'norma': 1, 'hoja_datos': 4}
📊 Con KV extraídos: 4

--- CHUNK ---
b. Cantidad por equipo según recomendado por el fabricante. c. Número de parte y catálogo del repuesto (drawing).
Metadatos: {'page': 19, 'content_hash': '379f10a10a3758d13d168c1f75220be7fcdae2d9', 'prio': 'norma', 'type': 'table', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'chunk_id': '19_2'}

--- CHUNK ---
Ítem | Cantidad | Unidad | TAG | Descripción | Código de Catálogo ECOPETROL 1 | 1 | UND | AX- 75317 | Patín de Preparación y Dosificación: Se requiere una unidad paquete de Preparación y Dosificación sobre patín, conformada al menos por: Un tanque horizontal de 1056 galones de acero inoxidable 304ss
Metadatos: {'kv': '{"material_tanque": "304SS", "api_675": "true"}', 'chunk_id': '6_1', 'content_hash': 'abddedb9bcd5fb14cf6781021c194697d9138c00', 'type': 'table', 'prio': 'hoja_datos', 'page': 6, 

In [22]:
# ============================================
# Paso 12 - Buscar en una página específica
# ============================================
def buscar_por_pagina(pagina: int, pregunta: str):
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": 5, "filter": {"page": pagina}}
    )
    results = retriever.get_relevant_documents(pregunta)

    print(f"🔎 Resultados en página {pagina}: {len(results)}")
    for r in results:
        print("\n--- CHUNK ---")
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)

# Ejemplo:
buscar_por_pagina(32, "alcance del suministro")
buscar_por_pagina(42, "caudal de la bomba")


🔎 Resultados en página 32: 5

--- CHUNK ---
componentes, o dieciocho (18) meses a partir de la entrega a satisfacción de estos por parte de ECOPETROL S.A. Durante este período la garantía debe cubrir la reparación o reemplazo de los componentes defectuosos por parte del VENDEDOR incluyendo todas las labores, equipos, transporte y demás costos generados y requeridos para llevar a cabo dicha reparación y/o reemplazo. 10. ANEXOS 10.1. LISTA DE
Metadatos: {'prio': 'hoja_datos', 'chunk_id': '32_3', 'page': 32, 'type': 'text', 'kv': '{"caudal_gph": "12-18", "presion_psig": "100", "hp_motor": "1", "material_tanque": "304SS", "api_675": "true"}', 'content_hash': 'f6b9a81d31c88735a0fc2c6c25cca415fbe916b5', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf'}

--- CHUNK ---
operacionales indicadas en la hoja de datos y demás documentos que hacen parte de esta requisición. El paquete de inyección de químicos, los quill´s de inyección retráctiles deberán ser suministrados

In [23]:
# ============================================
# Paso 13.1 – Inventario de sistemas (Skids, TAGs y cantidades)
# ============================================

import json

def prueba_inventario_sistemas():
    pregunta = "¿Cuáles son los skids o sistemas de dosificación requeridos? Dame los TAG y las cantidades."
    retriever = vectorstore.as_retriever(
        search_kwargs={"k": 10, "filter": {"type": "table"}}
    )
    results = retriever.get_relevant_documents(pregunta)

    print(f"🔎 Resultados prueba INVENTARIO (skids + tags + cantidades): {len(results)}")

    kv_count = 0
    for r in results:
        print("\n--- CHUNK INVENTARIO ---")
        print(r.page_content[:400].replace("\n", " "))
        print("Metadatos:", r.metadata)

        if "kv" in r.metadata:
            try:
                kv_dict = json.loads(r.metadata["kv"])
                print("KV dict:", kv_dict)
                kv_count += 1
            except Exception:
                print("KV (raw):", r.metadata["kv"])

    print(f"\n📊 Total de chunks con KV útiles: {kv_count}")

# Ejemplo de ejecución
prueba_inventario_sistemas()


🔎 Resultados prueba INVENTARIO (skids + tags + cantidades): 10

--- CHUNK INVENTARIO ---
95 96 97 98 | 2 | Se requiere una unidad paquete de Preparación y Dosificación sobre patín, conformada al menos por: Un tanque horizontal de 1056 galones de acero inoxidable 304ss con dos cámaras de preparación, y una camara para dosificación. Este paquete también incluye un sistema de control automático para la entrada de agua, para la introducción del polímero sólido, agitacion, alarma para vaci
Metadatos: {'prio': 'hoja_datos', 'content_hash': '7e2c5f057bb66d021301e0cd29b4dc6c3ca9a5c6', 'type': 'table', 'chunk_id': '43_3', 'kv': '{"caudal_gph": "12-18", "presion_psig": "100", "hp_motor": "1", "material_tanque": "304SS", "api_675": "true"}', 'page': 43, 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf'}
KV dict: {'caudal_gph': '12-18', 'presion_psig': '100', 'hp_motor': '1', 'material_tanque': '304SS', 'api_675': 'true'}

--- CHUNK INVENTARIO ---
-ASIGNACIÓN DE TAG A E

In [24]:
# ============================================
# Paso 14 (v3) – Alcance + Datos de diseño de la bomba API 675 (robusto + actualizado)
# ============================================
import re, json
from langchain.schema import Document

def _j(x):
    try:
        return json.loads(x)
    except Exception:
        return {}

def _parse_nums(text: str):
    """Extrae caudal (GPH), presión (psig) y HP desde texto libre (fallback)."""
    caudal = None
    pres = None
    hp = None

    # Caudal "12 - 18 GPH" | "12–18 Gph" | "12 a 18 GPH" | simple "18 GPH"
    m = re.search(r'(\d+(?:[.,]\d+)?)\s*(?:-|–|a|to)\s*(\d+(?:[.,]\d+)?)\s*(gph|g\.?p\.?h\.?)\b', text, re.I)
    if m:
        caudal = f"{m.group(1)} - {m.group(2)}"
    else:
        m = re.search(r'\b(\d+(?:[.,]\d+)?)\s*(gph|g\.?p\.?h\.?)\b', text, re.I)
        if m:
            caudal = m.group(1)

    # Presión "@ 100 psi" | "100 psig"
    m = re.search(r'@?\s*(\d+(?:[.,]\d+)?)\s*(psi|psig)\b', text, re.I)
    if m:
        pres = m.group(1)

    # Potencia "1 HP"
    m = re.search(r'\b(\d+(?:[.,]\d+)?)\s*hp\b', text, re.I)
    if m:
        hp = m.group(1)

    return caudal, pres, hp

def prueba_alcance_y_datos_bomba_v3():
    q_alc = "alcance del suministro paquete de inyección de químicos"
    q_hd  = "bomba dosificadora API 675 caudal presión HP materiales GPH psig"

    # 1) Recuperar alcance
    retr_alc = vectorstore.as_retriever(
        search_kwargs={"k": 12, "filter": {"prio": "alcance"}},
        search_type="mmr"
    )
    alc_docs = retr_alc.get_relevant_documents(q_alc)

    if not alc_docs:  # fallback
        retr_alc_fb = vectorstore.as_retriever(search_kwargs={"k": 12}, search_type="mmr")
        alc_docs = retr_alc_fb.get_relevant_documents("alcance del suministro")

    # 2) Recuperar hojas de datos
    retr_hd = vectorstore.as_retriever(
        search_kwargs={"k": 20, "filter": {"prio": "hoja_datos"}},
        search_type="mmr"
    )
    hd_docs = retr_hd.get_relevant_documents(q_hd)

    # 3) Refuerzo directo desde la colección (KV + texto clave)
    coll = vectorstore._collection
    meta_hd = coll.get(where={"prio": "hoja_datos"}, include=["metadatas", "documents"])
    for md, doc in zip(meta_hd["metadatas"], meta_hd["documents"]):
        if not isinstance(md, dict):
            continue
        if ("kv" in md and any(k in md["kv"] for k in ["caudal_gph", "presion_psig", "hp_motor", "material_tanque", "material_mojado", "bombas_operativas", "bombas_respaldo"])) \
           or ("API 675" in (doc or "") or "GPH" in (doc or "") or "psig" in (doc or "")):
            hd_docs.append(Document(page_content=doc, metadata=md))

    # 4) De-duplicar por (page,type,primeros 60 chars)
    def _key(d): return (d.metadata.get("page"), d.metadata.get("type"), (d.page_content or "")[:60])
    seen = set()
    alc_unique, hd_unique = [], []
    for d in alc_docs:
        k = _key(d)
        if k not in seen:
            seen.add(k); alc_unique.append(d)
    for d in hd_docs:
        k = _key(d)
        if k not in seen:
            seen.add(k); hd_unique.append(d)

    # 5) Extraer KV + regex → síntesis
    datos = {
        "caudal_gph": None,
        "presion_psig": None,
        "hp_motor": None,
        "materiales": set(),
        "api_675": False,
        "bombas_operativas": None,
        "bombas_respaldo": None,
        "paginas_fuente": set(),
    }

    for d in hd_unique:
        md = d.metadata or {}
        if "page" in md:
            datos["paginas_fuente"].add(md["page"])

        # KV directo
        if "kv" in md:
            kv = _j(md["kv"])
            if not datos["caudal_gph"] and kv.get("caudal_gph"):
                datos["caudal_gph"] = kv["caudal_gph"]
            if not datos["presion_psig"] and kv.get("presion_psig"):
                datos["presion_psig"] = kv["presion_psig"]
            if not datos["hp_motor"] and kv.get("hp_motor"):
                datos["hp_motor"] = kv["hp_motor"]
            if kv.get("material_tanque"):
                datos["materiales"].add(kv["material_tanque"])
            if kv.get("material_mojado"):
                datos["materiales"].add(kv["material_mojado"])
            if str(kv.get("api_675")).lower() == "true":
                datos["api_675"] = True
            if kv.get("bombas_operativas"):
                datos["bombas_operativas"] = kv["bombas_operativas"]
            if kv.get("bombas_respaldo"):
                datos["bombas_respaldo"] = kv["bombas_respaldo"]

        # Fallback regex
        if any(v is None for v in (datos["caudal_gph"], datos["presion_psig"], datos["hp_motor"])):
            c, p, h = _parse_nums(d.page_content or "")
            if c and not datos["caudal_gph"]:
                datos["caudal_gph"] = c
            if p and not datos["presion_psig"]:
                datos["presion_psig"] = p
            if h and not datos["hp_motor"]:
                datos["hp_motor"] = h

    # 6) Mostrar resultados
    print(f"🔎 ALCANCE (k={len(alc_unique)}):")
    for r in alc_unique[:5]:
        print(f"\n— p.{r.metadata.get('page')} | {r.metadata.get('type')}")
        print((r.page_content or "")[:350].replace("\n", " "))

    print(f"\n🔎 HOJA DE DATOS (k={len(hd_unique)}):")
    for r in hd_unique[:6]:
        print(f"\n— p.{r.metadata.get('page')} | {r.metadata.get('type')}")
        print((r.page_content or "")[:350].replace("\n", " "))
        if "kv" in r.metadata:
            print("KV:", r.metadata["kv"])

    print("\n✅ EXTRACCIÓN SINTETIZADA:")
    print(f"  • Caudal (GPH): {datos['caudal_gph']}")
    print(f"  • Presión (psig): {datos['presion_psig']}")
    print(f"  • Potencia (HP): {datos['hp_motor']}")
    print(f"  • Materiales: {', '.join(sorted(datos['materiales'])) or '—'}")
    print(f"  • API 675: {'Sí' if datos['api_675'] else 'N/D'}")
    print(f"  • Bombas operativas: {datos['bombas_operativas'] or '—'}")
    print(f"  • Bombas respaldo: {datos['bombas_respaldo'] or '—'}")
    print(f"  • Páginas fuente (HD): {sorted(datos['paginas_fuente'])}")

    return {"alcance": alc_unique, "hoja_datos": hd_unique, "datos": datos}

# Ejecución:
prueba_alcance_y_datos_bomba_v3()



🔎 ALCANCE (k=12):

— p.1 | text
MAYORES PAQUETES DE INYECCIÓN DE QUÍMICOS

— p.64 | text
PAQUETE de inyección de químicos. Por lo que debe suministrar todos los diagramas de CAS09991MERET000002_InyeccionQxSTAPEC3_AFC2.doc 8 de 18

— p.73 | text
de Inyección suministradas serán las estipuladas en la orden de compra, sin embargo, para efectos de la cotización el PROVEEDOR deberá tener en cuenta que la garantía debe ser como mínimo de 12 meses a partir de la puesta en marcha del PAQUETE de inyección de químicos y los Quill´s de Inyección o 18 meses a partir de la fecha de entrega a satisfacc

— p.31 | text
El paquete de inyección de químicos y los quill´s de inyección retráctiles, junto con todos sus componentes, deben ser garantizados por el VENDEDOR contra fallas de diseño, materiales defectuosos o inapropiados, defectos de fabricación y fallas producidas en condiciones normales de operación. El VENDEDOR debe cumplir con las condiciones de operació

— p.62 | text
ser suministrada con la

{'alcance': [Document(id='7b51cb13365437e7dc0ce2c105ef1e958e5a9eec', metadata={'page': 1, 'content_hash': '7b51cb13365437e7dc0ce2c105ef1e958e5a9eec', 'prio': 'alcance', 'type': 'text', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'chunk_id': '1_1'}, page_content='MAYORES\nPAQUETES DE INYECCIÓN DE QUÍMICOS'),
  Document(id='d594fb8cee2f02eb809445e7a50d06333d1b1586', metadata={'page': 64, 'chunk_id': '64_5', 'content_hash': 'd594fb8cee2f02eb809445e7a50d06333d1b1586', 'type': 'text', 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'prio': 'alcance'}, page_content='PAQUETE de inyección de químicos. Por lo que debe suministrar todos los diagramas de\nCAS09991MERET000002_InyeccionQxSTAPEC3_AFC2.doc 8 de 18'),
  Document(id='069d996f4306cf0462ca9e456f87b04a5feaf62e', metadata={'chunk_id': '73_3', 'type': 'text', 'page': 73, 'source': './data/CAS09991MERMR000003_InyeccionQxSTAPEC3_4_250429_200355.pdf', 'content_hash': '069d996f43